# Group Survey Data Analysis and Cleaning

Analysis of ethnic group survey responses.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image, display

### Load the Data

In [ ]:
# Load the raw data form the survey you just participated in at https://survey.hcnmplatform.org/
df = pd.read_csv('results-survey335431.csv')

### Exploring and Cleaning

In [ ]:
df.info()

In [ ]:
df_clean = df.copy()

In [ ]:
column_mapping = {}
for col in df_clean.columns:
    if 'Which ethnic gr' in col:
        if '[Moldovan]' in col:
            column_mapping[col] = 'Moldovan'
        elif '[Romanian]' in col:
            column_mapping[col] = 'Romanian'
        elif '[Ukrainian]' in col:
            column_mapping[col] = 'Ukrainian'
        elif '[Russian]' in col:
            column_mapping[col] = 'Russian'
        elif '[Gagauz]' in col:
            column_mapping[col] = 'Gagauz'
        elif '[Bulgarian]' in col:
            column_mapping[col] = 'Bulgarian'
        elif '[Roma]' in col:
            column_mapping[col] = 'Roma'
        elif '[Prefer not to a' in col:
            column_mapping[col] = 'Prefer_not_to_answer'
        elif '[Other]' in col:
            column_mapping[col] = 'Other'

df_clean = df_clean.rename(columns=column_mapping)

In [ ]:
df_clean.columns

In [ ]:
df_clean = df_clean.loc[~df_clean['Date submitted'].isna()]

In [ ]:
df_clean = df_clean.iloc[:, 5:]

In [ ]:
df_clean.info()

In [ ]:
df_clean['Other'].value_counts()

In [ ]:
display(Image(filename='ferenghi.jpg', width=500))

In [ ]:
df_clean['Other'] = df_clean['Other'].apply(
    lambda x: 'Yes' if pd.notna(x) and x != '' else 'No')

In [ ]:
# # Convert Yes/No to binary (1/0) for analysis
# for col in df_clean:
#     if col in df_clean.columns:
#         df_clean.loc[:, f'{col}_binary'] = df_clean[col].map({'Yes': 1, 'No': 0})

# binary_cols = [col for col in df_clean.columns if '_binary' in col]

## 4. Analysis

In [ ]:
df_clean.info()

In [ ]:
df_clean

In [ ]:
df_clean.apply(lambda row: (row == 'Yes').sum(), axis=1)

In [ ]:
df_clean.apply(lambda col: (col == 'Yes').sum()).plot(kind='bar')